## ROS OpenCV

In real applications a robot has to take information of surrounding.

A very useful method is from Vision.

OpenCV is a worldwide used Open Source Computer Vision Library

Image processing and computer vision are key disciplines for this objective.

### OpenCV install

In [ ]:
sudo apt-get update
sudo apt-get install ros-melodic-vision-opencv

In ROS-kinetic type: sudo apt-get install ros-kinetic-opencv3

Sometimes in Melodic has to use:

In [ ]:
sudo apt-get install python-opencv

In principle, this should work.

In case you face a problem with imshow (as I faced a similar one), you may try the following solution (use pip3 if you use Python 3)

In [ ]:
sudo apt-get install libopencv-*
pip install opencv-contrib-python

To verify the openCV installation type in a terminal:

In [ ]:
python -c "import cv2; print(cv2.__version__)"

The output will be the version installed (3.2.0)

If you are in a python shell, typr "import cv2" and if there is no error, all is OK. You can exit typing: exit()

You will need to install numpy and matplotlib python libraries:


In [ ]:
sudo apt install python-pip
pip install numpy
pip install matplotlib

#### USB camera install

To install a USB cam type

In [ ]:
sudo apt-get install ros-melodic-usb-cam

Be sure to have installed the "image-view" package

In [ ]:
sudo apt-get install ros-melodic-image-view

Install the camera in your Virtual machine:
- From Virtual Box download webpage: https://www.virtualbox.org/wiki/Downloads
- download the irtualBox 6.1.16 Oracle VM VirtualBox Extension Pack

<img src="./Images/1_USB.png">

- From VirtualBox/preferences/extensions add a new extension and select the Extension Pack file previously downloaded

<img src="./Images/1_USB1.png">

- close the VirtualBox and open it again
- open the Virtual machine and select the webcam from top menu

<img src="./Images/1_USB2.png">

### CvBridge: Bridging OpenCV and ROS

The format of images in OpenCV and ROS are different. 

http://wiki.ros.org/cv_bridge/Tutorials/ConvertingBetweenROSImagesAndOpenCVImagesPython

<img src="./Images/1_CvBridge.png">

#### Vision package

We will create in our workspace a "vision" package where use OpenCV in ROS to perform Image processing.

- In VS Code, open the rubot_opencv_ws workspace and first create a "vision" package with dependencies:

In [ ]:
catkin_create_pkg vision std_msgs sensor_msgs cv_bridge rospy

- compile the ws

In [ ]:
cd ~/rubot_opencv_ws
catkin_make

- verify the .bashrc file
- add in src folder the python files for image processing

The first python file we will add is the "image_pub_sub.py" file to read a ROS image message and convert it into OpenCV format

In this file, a node is created that listens to a ROS image message topic, converts the images into an cv::Mat OpenCV format, draws a circle on it and displays the image using OpenCV. The image is then republished over ROS.
This file can be adapted for the Image origin camera:
- Turtlebot3 simulated cam
- USB cam
- gopigo3 cam

In [ ]:
#!/usr/bin/env python

import rospy
import cv2
from std_msgs.msg import String
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError
import sys

bridge = CvBridge()

def image_callback(ros_image):
  print 'got an image'
  global bridge
  #convert ros_image into an opencv-compatible image
  try:
    cv_image = bridge.imgmsg_to_cv2(ros_image, "bgr8")
  except CvBridgeError as e:
      print(e)
  #from now on, you can work exactly like with opencv
  (rows,cols,channels) = cv_image.shape
  if cols > 200 and rows > 200 :
      cv2.circle(cv_image, (100,100),90, 255)
  font = cv2.FONT_HERSHEY_SIMPLEX
  cv2.putText(cv_image,'Webcam Activated with ROS & OpenCV!',(10,350), font, 1,(255,255,255),2,cv2.LINE_AA)
  cv2.imshow("Image window", cv_image)
  cv2.waitKey(3)

  
def main(args):
  rospy.init_node('image_converter', anonymous=True)
  #for turtlebot3 waffle
  #image_topic="/camera/rgb/image_raw/compressed"
  #for usb cam
  #image_topic="/usb_cam/image_raw"
  image_sub = rospy.Subscriber("/camera/rgb/image_raw/compressed",Image, image_callback)
  try:
    rospy.spin()
  except KeyboardInterrupt:
    print("Shutting down")
  cv2.destroyAllWindows()

if __name__ == '__main__':
    main(sys.argv)

We will test it in these 3 different environmnets:
- turtlebot3 environment in turtlebot3_pc_ws workspace
- USB Cam device
- gopigo3 robot in gopigo3_rbpi_ws

##### Turtlebot3 environment

- In VS Code, clone the turtlebot3_simulations package in src folder of our workspace

In [ ]:
cd src
git clone https://github.com/ROBOTIS-GIT/turtlebot3_simulations

- delete the .git folder inside the turtlebot3_simulations package if you want to syncronise this package in your repository
- compile your workspace

- add the "tb3_image_pub_sub.py" file in src folder of "vision" package. In this file we take the corresponding topic name of ROS image

In [ ]:
#!/usr/bin/env python

import rospy
import cv2
from std_msgs.msg import String
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError
import sys

bridge = CvBridge()

def image_callback(ros_image):
  print 'got an image'
  global bridge
  #convert ros_image into an opencv-compatible image
  try:
    cv_image = bridge.imgmsg_to_cv2(ros_image, "bgr8")
  except CvBridgeError as e:
      print(e)
  #from now on, you can work exactly like with opencv
  (rows,cols,channels) = cv_image.shape
  if cols > 200 and rows > 200 :
      cv2.circle(cv_image, (100,100),90, 255)
  font = cv2.FONT_HERSHEY_SIMPLEX
  cv2.putText(cv_image,'Webcam Activated with ROS & OpenCV!',(10,350), font, 1,(255,255,255),2,cv2.LINE_AA)
  cv2.imshow("Image window", cv_image)
  cv2.waitKey(3)

  
def main(args):
  rospy.init_node('image_converter', anonymous=True)
  #for turtlebot3 waffle
  #image_topic="/camera/rgb/image_raw"
  #for usb cam
  #image_topic="/usb_cam/image_raw"
  image_sub = rospy.Subscriber("/camera/rgb/image_raw",Image, image_callback)
  try:
    rospy.spin()
  except KeyboardInterrupt:
    print("Shutting down")
  cv2.destroyAllWindows()

if __name__ == '__main__':
    main(sys.argv)

- spawn the turtle3 robot in a physical environment and publish the image of the simulated camera with a sensor_msgs/Image message type in /camera/rgb/image_raw topic.

In [ ]:
roslaunch turtlebot3_gazebo turtlebot3_world.launch

In [ ]:
rosrun vision tb3_image_pub_sub.py

<img src="./Images/1_tb3cam.png">

#### USB cam device

- select in "Devices" top menu the USB camera
- add the "USBcam_image_pub_sub.py" file in src folder of "vision" package. In this file we take the corresponding topic name of ROS image

In [ ]:
roscore

To publish the image uses the "usb_cam" package already installed: http://wiki.ros.org/usb_cam

In [ ]:
rosrun usb_cam usb_cam_node _pixel_format:=yuyv

- To verify if the USB cam is working, use "image_view" package: http://wiki.ros.org/image_view

In [ ]:
rosrun image_view image_view image:=/usb_cam/image_raw _image_transport:=raw

<img src="./Images/1_USB3.png">

- You can run our python program

In [ ]:
cd ~/rubot_opencv_ws
rosrun vision USBcam_image_pub_sub.py 

<img src="./Images/1_USB4.png">

#### gopigo3 robot

In gopigo3_rbpi_ws you have all the packages needed for OpenCV.
You have created a "autonomous_robots" package with "script" folder.
copy the "image_pub_sub.py" file.

run a launch file to open raspicam 

In [ ]:
roslaunch raspicam_node camerav2_1280x960_10fps.launch enable_raw:=true camera_frame_id:="laser_frame"

run now the python file

In [ ]:
rosrun autonomous_robots image_pub_sub.py